<a href="https://colab.research.google.com/github/arminak6/Control-Quality/blob/Dev/Code/anomalib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anomalib

In [2]:
!pip install lightning
!pip install kornia
!pip install FrEIA
!pip install python-dotenv
!pip install open-clip-torch
!pip install openvino
!pip install openvino-dev
!pip install onnx

  Using cached kornia-0.7.4-py2.py3-none-any.whl.metadata (18 kB)
  Using cached kornia_rs-0.1.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.1 kB)
Using cached kornia-0.7.4-py2.py3-none-any.whl (899 kB)
Using cached kornia_rs-0.1.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached FrEIA-0.2.tar.gz (34 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for FrEIA: filename=FrEIA-0.2-py3-none-any.whl size=42762 sha256=a808b57ba42a660907acebff96b1baea31ca93dd974c61b5da6c28da3df71f07
  Stored in directory: /root/.cache/pip/wheels/81/a8/e2/d532a76f72108ac4a340cbe3f86b4f591abfdbd75209a5badb
Successfully built FrEIA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 86.4 MB/s eta 0:00:00
   

In [4]:
from anomalib.data import Folder
from anomalib.models import EfficientAd
from anomalib.engine import Engine
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import CSVLogger
import pandas as pd
import matplotlib.pyplot as plt

# Dataset Configuration
datamodule = Folder(
    name="custom_dataset",
    root="/content/drive/MyDrive/praeciso/anomalib_dataset",
    normal_dir="train/good",
    abnormal_dir="test/anomalous",
    extensions=[".bmp"],
    task="classification"
)

# Set batch sizes
datamodule.train_batch_size = 1
datamodule.eval_batch_size = 1

# Initialize Model and Callbacks
model = EfficientAd()

# Define Model Checkpoint Callback
checkpoint_callback = ModelCheckpoint(
    monitor="image_AUROC",  # Save the model with the highest AUROC
    mode="max",
    save_top_k=1
)

# Define Early Stopping Callback
early_stopping_callback = EarlyStopping(
    monitor="image_AUROC",  # Monitor AUROC instead of val_loss
    patience=5,
    mode="max",
    verbose=True
)

# Define CSV Logger for Logging Metrics
csv_logger = CSVLogger("logs", name="EfficientAd_training")

# Initialize Engine with Callbacks and Logger
engine = Engine(
    max_epochs=50,
    accelerator="gpu",
    devices=1,
    callbacks=[checkpoint_callback, early_stopping_callback],
    logger=csv_logger
)

# Train the Model
engine.fit(datamodule=datamodule, model=model)


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: 
  | Name                  | 

Training: |          | 0/? [00:00<?, ?it/s]


efficientad_pretrained_weights.zip: 0.00B [00:00, ?B/s]
efficientad_pretrained_weights.zip:   0%|          | 8.19k/40.0M [00:01<2:06:49, 5.25kB/s]
efficientad_pretrained_weights.zip:  26%|██▋       | 10.5M/40.0M [00:01<00:03, 7.58MB/s]  
efficientad_pretrained_weights.zip:  53%|█████▎    | 21.0M/40.0M [00:02<00:01, 14.3MB/s]
efficientad_pretrained_weights.zip: 40.0MB [00:02, 16.5MB/s]                            
/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/lightning_model.py:98: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. A

Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:05<00:00,  3.56it/s]
INFO: Metric image_AUROC improved. New best score: 0.583
INFO:lightning.pytorch.callbacks.early_stopping:Metric image_AUROC improved. New best score: 0.583


...

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:04<00:00,  4.21it/s]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:04<00:00,  3.93it/s]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:02<00:00,  6.83it/s]
INFO: Monitored metric image_AUROC did not improve in the last 5 records. Best score: 1.000. Signaling Trainer to stop.
INFO:lightning.pytorch.callbacks.early_stopping:Monitored metric image_AUROC did not improve in the last 5 records. Best score: 1.000. Signaling Trainer to stop.


In [7]:
# Evaluate the best checkpoint after training
test_results = engine.test(
    model=model,
    datamodule=datamodule,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path
)

print("Test Results:", test_results)

/usr/local/lib/python3.10/dist-packages/anomalib/engine/engine.py:391: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=model.de

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │     0.95652174949646      │
└───────────────────────────┴───────────────────────────┘

Test Results: [{'image_AUROC': 1.0, 'image_F1Score': 0.95652174949646}]


In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Directory to save plots
output_dir = "/content/drive/MyDrive/praeciso/show2"
os.makedirs(output_dir, exist_ok=True)

# Check logger directory and log file
log_file = os.path.join(csv_logger.log_dir, "metrics.csv")
if not os.path.exists(log_file):
    raise FileNotFoundError(f"Log file not found: {log_file}")

# Load training logs
logs = pd.read_csv(log_file)

# Check available metrics
print("Available metrics in logs:")
print(logs.columns)

# Plot AUROC if available
if "image_AUROC" in logs:
    plt.figure(figsize=(10, 5))
    plt.plot(logs["epoch"], logs["image_AUROC"], label="Image AUROC", marker="o")
    plt.xlabel("Epoch")
    plt.ylabel("AUROC")
    plt.title("Image AUROC During Training")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, "image_auroc.png"))  # Save plot
    plt.close()  # Close the figure

# Plot F1 Score if available
if "image_F1Score" in logs:
    plt.figure(figsize=(10, 5))
    plt.plot(logs["epoch"], logs["image_F1Score"], label="Image F1 Score", marker="o")
    plt.xlabel("Epoch")
    plt.ylabel("F1 Score")
    plt.title("Image F1 Score During Training")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, "image_f1_score.png"))  # Save plot
    plt.close()  # Close the figure

# Plot train_loss or val_loss if available
if "train_loss" in logs or "val_loss" in logs:
    plt.figure(figsize=(10, 5))
    if "train_loss" in logs:
        plt.plot(logs["epoch"], logs["train_loss"], label="Training Loss", marker="o")
    if "val_loss" in logs:
        plt.plot(logs["epoch"], logs["val_loss"], label="Validation Loss", marker="o")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, "loss_curve.png"))  # Save plot
    plt.close()  # Close the figure

print(f"Plots saved successfully in {output_dir}")


Available metrics in logs:
Index(['epoch', 'image_AUROC', 'image_F1Score', 'step', 'train_ae_epoch',
       'train_ae_step', 'train_loss_epoch', 'train_loss_step',
       'train_st_epoch', 'train_st_step', 'train_stae_epoch',
       'train_stae_step'],
      dtype='object')
Plots saved successfully in /content/drive/MyDrive/praeciso/show2


In [9]:

import os
import matplotlib.pyplot as plt

# Directory to save plots
output_dir = "/content/drive/MyDrive/praeciso/show2"
os.makedirs(output_dir, exist_ok=True)

# Check available metrics and handle missing data
if "epoch" in logs.columns:
    # Filter rows with non-NaN values for AUROC
    if "image_AUROC" in logs:
        valid_logs_auroc = logs[~logs["image_AUROC"].isna()]
        if not valid_logs_auroc.empty:
            plt.figure(figsize=(10, 5))
            plt.plot(valid_logs_auroc["epoch"], valid_logs_auroc["image_AUROC"], label="Image AUROC", marker="o")
            plt.xlabel("Epoch")
            plt.ylabel("AUROC")
            plt.title("Image AUROC During Training")
            plt.legend()
            plt.grid()
            plt.savefig(os.path.join(output_dir, "image_auroc_fixed.png"))
            plt.close()
        else:
            print("No valid data for Image AUROC found in logs.")

    # Filter rows with non-NaN values for F1 Score
    if "image_F1Score" in logs:
        valid_logs_f1 = logs[~logs["image_F1Score"].isna()]
        if not valid_logs_f1.empty:
            plt.figure(figsize=(10, 5))
            plt.plot(valid_logs_f1["epoch"], valid_logs_f1["image_F1Score"], label="Image F1 Score", marker="o")
            plt.xlabel("Epoch")
            plt.ylabel("F1 Score")
            plt.title("Image F1 Score During Training")
            plt.legend()
            plt.grid()
            plt.savefig(os.path.join(output_dir, "image_f1_score_fixed.png"))
            plt.close()
        else:
            print("No valid data for Image F1 Score found in logs.")

    # Plot training loss if available
    if "train_loss_step" in logs:
        valid_logs_loss = logs[~logs["train_loss_step"].isna()]
        if not valid_logs_loss.empty:
            plt.figure(figsize=(10, 5))
            plt.plot(valid_logs_loss["epoch"], valid_logs_loss["train_loss_step"], label="Training Loss", marker="o")
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.title("Training Loss During Training")
            plt.legend()
            plt.grid()
            plt.savefig(os.path.join(output_dir, "train_loss_fixed.png"))
            plt.close()
        else:
            print("No valid data for Training Loss found in logs.")
else:
    print("`epoch` column not found in logs.")

print(f"Plots saved successfully in {output_dir}")

Plots saved successfully in /content/drive/MyDrive/praeciso/show2


In [10]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

# Directory to save outputs
output_dir = "combined_visualizations"
os.makedirs(output_dir, exist_ok=True)

# Set threshold for segmentation
threshold_value = 0.5  # Adjust this based on your data

# Set the number of images to process
num_samples = 10

# Randomly select images from defected dataset
test_loader = datamodule.test_dataloader()
defected_images = []
defected_predictions = []

for batch, prediction in zip(test_loader, engine.trainer.predict(model=model, dataloaders=test_loader)):
    labels = batch["label"].detach().cpu().numpy()  # Get labels (0: normal, 1: anomalous)
    for i in range(len(labels)):
        if labels[i] == 1:  # Anomalous images only
            defected_images.append(batch["image"][i])
            defected_predictions.append(prediction["anomaly_maps"][i])

# Randomly select 10 images if available
random_indices = random.sample(range(len(defected_images)), min(num_samples, len(defected_images)))
selected_images = [defected_images[i] for i in random_indices]
selected_predictions = [defected_predictions[i] for i in random_indices]

# Process each selected image
for idx, (image, anomaly_map) in enumerate(zip(selected_images, selected_predictions)):
    unique_id = f"{idx+1:03d}"  # Unique number for each image

    # Normalize anomaly map
    anomaly_map = anomaly_map.squeeze().detach().cpu().numpy()
    anomaly_map = (anomaly_map - anomaly_map.min()) / (anomaly_map.max() - anomaly_map.min())

    # Convert image for visualization
    image_np = image.permute(1, 2, 0).cpu().numpy()
    image_np = (image_np * 255).astype(np.uint8)

    # Create heatmap
    anomaly_map_resized = cv2.resize(anomaly_map, (image_np.shape[1], image_np.shape[0]))
    heatmap = cv2.applyColorMap((anomaly_map_resized * 255).astype(np.uint8), cv2.COLORMAP_JET)
    heatmap_overlay = cv2.addWeighted(image_np, 0.7, heatmap, 0.3, 0)

    # Create segmentation mask
    binary_mask = (anomaly_map > threshold_value).astype(np.uint8)
    binary_mask_resized = cv2.resize(binary_mask, (image_np.shape[1], image_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmentation_overlay = cv2.addWeighted(image_np, 0.7, cv2.applyColorMap(binary_mask_resized * 255, cv2.COLORMAP_JET), 0.3, 0)

    # Apply colormap to anomaly map before saving
    anomaly_map_colored = cv2.applyColorMap((anomaly_map_resized * 255).astype(np.uint8), cv2.COLORMAP_JET)

    # Save all outputs
    cv2.imwrite(os.path.join(output_dir, f"image_{unique_id}.png"), cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))
    cv2.imwrite(os.path.join(output_dir, f"anomaly_map_{unique_id}.png"), anomaly_map_colored)  # Save as colored map
    cv2.imwrite(os.path.join(output_dir, f"heatmap_{unique_id}.png"), heatmap_overlay)
    cv2.imwrite(os.path.join(output_dir, f"segmentation_{unique_id}.png"), segmentation_overlay)


    # Optional: Display outputs
    plt.figure(figsize=(20, 5))

    plt.subplot(1, 4, 1)
    plt.title("Original Image")
    plt.imshow(image_np)
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.title("Anomaly Map")
    plt.imshow(anomaly_map, cmap="jet")
    plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.title("Heatmap Overlay")
    plt.imshow(heatmap_overlay)
    plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.title("Segmentation Map")
    plt.imshow(segmentation_overlay)
    plt.axis("off")

    plt.show()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [11]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

# Directory to save outputs
parent_dir = "/content/drive/MyDrive/praeciso/show2"
output_dir = os.path.join(parent_dir, "combined_visualizations")
os.makedirs(output_dir, exist_ok=True)

# Set threshold for segmentation
threshold_value = 0.5  # Adjust this based on your data

# Set the number of images to process
num_samples = 10

# Randomly select images from defected dataset
test_loader = datamodule.test_dataloader()
defected_images = []
defected_predictions = []

for batch, prediction in zip(test_loader, engine.trainer.predict(model=model, dataloaders=test_loader)):
    labels = batch["label"].detach().cpu().numpy()  # Get labels (0: normal, 1: anomalous)
    for i in range(len(labels)):
        if labels[i] == 1:  # Anomalous images only
            defected_images.append(batch["image"][i])
            defected_predictions.append(prediction["anomaly_maps"][i])

# Randomly select images
random_indices = random.sample(range(len(defected_images)), min(num_samples, len(defected_images)))
selected_images = [defected_images[i] for i in random_indices]
selected_predictions = [defected_predictions[i] for i in random_indices]

# Process each selected image
for idx, (image, anomaly_map) in enumerate(zip(selected_images, selected_predictions)):
    unique_id = f"{idx+1:03d}"  # Unique identifier for each image

    # Normalize anomaly map
    anomaly_map = anomaly_map.squeeze().detach().cpu().numpy()
    anomaly_map = (anomaly_map - anomaly_map.min()) / (anomaly_map.max() - anomaly_map.min())

    # Convert image for visualization
    image_np = image.permute(1, 2, 0).cpu().numpy()
    image_np = (image_np * 255).astype(np.uint8)

    # Create heatmap
    anomaly_map_resized = cv2.resize(anomaly_map, (image_np.shape[1], image_np.shape[0]))
    heatmap = cv2.applyColorMap((anomaly_map_resized * 255).astype(np.uint8), cv2.COLORMAP_JET)
    heatmap_overlay = cv2.addWeighted(image_np, 0.7, heatmap, 0.3, 0)

    # Create segmentation mask
    binary_mask = (anomaly_map > threshold_value).astype(np.uint8)
    binary_mask_resized = cv2.resize(binary_mask, (image_np.shape[1], image_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmentation_overlay = cv2.addWeighted(image_np, 0.7, cv2.applyColorMap(binary_mask_resized * 255, cv2.COLORMAP_JET), 0.3, 0)

    # Combine all visuals into one image
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    axes[0].imshow(image_np)
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    axes[1].imshow(anomaly_map, cmap="jet")
    axes[1].set_title("Anomaly Map")
    axes[1].axis("off")

    axes[2].imshow(heatmap_overlay)
    axes[2].set_title("Heatmap Overlay")
    axes[2].axis("off")

    axes[3].imshow(segmentation_overlay)
    axes[3].set_title("Segmentation Map")
    axes[3].axis("off")

    # Save combined image
    combined_image_path = os.path.join(output_dir, f"combined_visualization_{unique_id}.png")
    plt.savefig(combined_image_path)
    plt.close(fig)

print(f"Combined visualizations saved in {output_dir}")


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Combined visualizations saved in /content/drive/MyDrive/praeciso/show2/combined_visualizations
